In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

_data = pd.read_csv('./Covid Data.csv')
print(_data.shape)
_data[:5]

(1048575, 21)


,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97


In [19]:
_data.dtypes

USMER                     int64
MEDICAL_UNIT              int64
SEX                       int64
PATIENT_TYPE              int64
INTUBED                   int64
PNEUMONIA                 int64
AGE                       int64
PREGNANT                  int64
DIABETES                  int64
COPD                      int64
ASTHMA                    int64
INMSUPR                   int64
HIPERTENSION              int64
OTHER_DISEASE             int64
CARDIOVASCULAR            int64
OBESITY                   int64
RENAL_CHRONIC             int64
TOBACCO                   int64
CLASIFFICATION_FINAL      int64
ICU                       int64
DIED                    float64
dtype: object

In [ ]:
_data["SEX"] = _data.SEX.map({1: 0, 2: 1})

In [ ]:
dd_condition = _data["DATE_DIED"] == "9999-99-99"
_data.loc[dd_condition, "DIED"] = 0
_data.loc[~dd_condition, "DIED"] = 1
_data["DIED"] = _data.DIED.astype(int)
_data.drop("DATE_DIED", inplace=True, axis=1)

In [14]:
_data.INTUBED.value_counts()

INTUBED
97    848544
2     159050
1      33656
99      7325
Name: count, dtype: int64

In [18]:
for d in _data.columns:
    print(_data[d].value_counts().sort_index())
    print()

USMER
1    385672
2    662903
Name: count, dtype: int64

MEDICAL_UNIT
1        151
2        169
3      19175
4     314405
5       7244
6      40584
7        891
8      10399
9      38116
10      7873
11      5577
12    602995
13       996
Name: count, dtype: int64

SEX
0    525064
1    523511
Name: count, dtype: int64

PATIENT_TYPE
1    848544
2    200031
Name: count, dtype: int64

INTUBED
1      33656
2     159050
97    848544
99      7325
Name: count, dtype: int64

PNEUMONIA
1     140038
2     892534
99     16003
Name: count, dtype: int64

AGE
0      3862
1      4802
2      3178
3      2559
4      2485
       ... 
117       3
118       2
119       3
120       5
121       1
Name: count, Length: 121, dtype: int64

PREGNANT
1       8131
2     513179
97    523511
98      3754
Name: count, dtype: int64

DIABETES
1     124989
2     920248
98      3338
Name: count, dtype: int64

COPD
1       15062
2     1030510
98       3003
Name: count, dtype: int64

ASTHMA
1       31572
2     1014024
98  

In [20]:
_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 21 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   USMER                 1048575 non-null  int64  
 1   MEDICAL_UNIT          1048575 non-null  int64  
 2   SEX                   1048575 non-null  int64  
 3   PATIENT_TYPE          1048575 non-null  int64  
 4   INTUBED               1048575 non-null  int64  
 5   PNEUMONIA             1048575 non-null  int64  
 6   AGE                   1048575 non-null  int64  
 7   PREGNANT              1048575 non-null  int64  
 8   DIABETES              1048575 non-null  int64  
 9   COPD                  1048575 non-null  int64  
 10  ASTHMA                1048575 non-null  int64  
 11  INMSUPR               1048575 non-null  int64  
 12  HIPERTENSION          1048575 non-null  int64  
 13  OTHER_DISEASE         1048575 non-null  int64  
 14  CARDIOVASCULAR        1048575 non-

In [21]:
def clean_covid_df(df, drop_leaky=False, drop_high_missing=False, missing_thresh=0.4):
    df = df.copy()

    # 1) استانداردسازی برچسب هدف
    if "DIED" in df.columns:
        df = df[df["DIED"].notna()].copy()
        df["DIED"] = df["DIED"].astype("uint8")
    else:
        raise ValueError("Column 'DIED' not found.")

    # 2) نگاشت مقادیر نامشخص و باینری‌ها
    unknown_codes = {97: np.nan, 98: np.nan, 99: np.nan}
    bin_yesno = {1: 1, 2: 0}

    bin_cols = [
        "USMER",
        "INTUBED",
        "PNEUMONIA",
        "PREGNANT",
        "DIABETES",
        "COPD",
        "ASTHMA",
        "INMSUPR",
        "HIPERTENSION",
        "OTHER_DISEASE",
        "CARDIOVASCULAR",
        "OBESITY",
        "RENAL_CHRONIC",
        "TOBACCO",
        "ICU",
    ]
    for c in bin_cols:
        if c in df.columns:
            df[c] = df[c].replace(unknown_codes)
            df[c] = df[c].map(bin_yesno)
            # اندیکاتور نامشخص
            df[f"{c}_unk"] = df[c].isna().astype("uint8")
            # نوع داده با پشتیبانی از NA
            df[c] = df[c].astype("Int8")

    # 3) ستون‌های ویژه
    # SEX: همان 0/1 موجود را نگه داریم
    if "SEX" in df.columns:
        # اگر SEX مقدار دیگری جز 0/1 دارد، به NaN تبدیل شود
        df.loc[~df["SEX"].isin([0, 1]), "SEX"] = np.nan
        df["SEX_unk"] = df["SEX"].isna().astype("uint8")
        df["SEX"] = df["SEX"].astype("Int8")

    # PATIENT_TYPE: چنددسته‌ای (1/2). نگه داشتن به عنوان عدد صحیح/دسته‌ای
    if "PATIENT_TYPE" in df.columns:
        df["PATIENT_TYPE"] = df["PATIENT_TYPE"].astype("Int8")

    # MEDICAL_UNIT: دسته‌ای
    if "MEDICAL_UNIT" in df.columns:
        df["MEDICAL_UNIT"] = df["MEDICAL_UNIT"].astype("category")

    # CLASIFFICATION_FINAL: دسته‌ای
    if "CLASIFFICATION_FINAL" in df.columns:
        df["CLASIFFICATION_FINAL"] = df["CLASIFFICATION_FINAL"].astype("category")

    # 4) سن: پاک‌سازی دامنه و نوع
    if "AGE" in df.columns:
        df["AGE"] = pd.to_numeric(df["AGE"], errors="coerce")
        # کلیپ در بازه منطقی
        df["AGE"] = df["AGE"].clip(lower=0, upper=110)
        df["AGE_unk"] = df["AGE"].isna().astype("uint8")
        # NaN ها را با میانه پر نکن؛ اجازه بده مدل/ایمپیوتِر تصمیم بگیرد
        # df["AGE"] = df["AGE"].fillna(df["AGE"].median()).astype("int16")

    # 5) سازگاری منطقی: بارداری در سن‌های ناممکن
    if "PREGNANT" in df.columns and "AGE" in df.columns:
        impossible_age_mask = (df["AGE"] < 10) | (df["AGE"] > 55)
        # فقط NaN یا 1 را روی این سن‌ها به 0 تبدیل کن
        df.loc[impossible_age_mask & df["PREGNANT"].isna(), "PREGNANT"] = 0
        df.loc[impossible_age_mask & (df["PREGNANT"] == 1), "PREGNANT"] = 0
        df["PREGNANT"] = df["PREGNANT"].astype("Int8")

    # 6) نشت اطلاعات (اختیاری)
    if drop_leaky:
        to_drop = [c for c in ["INTUBED", "ICU"] if c in df.columns]
        df = df.drop(
            columns=to_drop + [f"{c}_unk" for c in to_drop if f"{c}_unk" in df.columns]
        )

    # 7) حذف ستون‌های با گمشدگی بسیار زیاد (اختیاری)
    if drop_high_missing:
        miss_ratio = df.isna().mean()
        high_miss_cols = miss_ratio[miss_ratio > missing_thresh].index.tolist()
        df = df.drop(columns=high_miss_cols)

    # 8) بهینه‌سازی انواع داده
    small_int_cols = ["USMER", "PATIENT_TYPE", "SEX", "DIED"]
    for c in small_int_cols:
        if c in df.columns and pd.api.types.is_integer_dtype(df[c]):
            # DIED باید بدون NA باشد
            if c == "DIED":
                df[c] = df[c].astype("uint8")
            else:
                df[c] = df[c].astype("Int8")

    return df

In [30]:
clean_data = clean_covid_df(_data)
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 38 columns):
 #   Column                Non-Null Count    Dtype   
---  ------                --------------    -----   
 0   USMER                 1048575 non-null  Int8    
 1   MEDICAL_UNIT          1048575 non-null  category
 2   SEX                   1048575 non-null  Int8    
 3   PATIENT_TYPE          1048575 non-null  Int8    
 4   INTUBED               192706 non-null   Int8    
 5   PNEUMONIA             1032572 non-null  Int8    
 6   AGE                   1048575 non-null  int64   
 7   PREGNANT              652955 non-null   Int8    
 8   DIABETES              1045237 non-null  Int8    
 9   COPD                  1045572 non-null  Int8    
 10  ASTHMA                1045596 non-null  Int8    
 11  INMSUPR               1045171 non-null  Int8    
 12  HIPERTENSION          1045471 non-null  Int8    
 13  OTHER_DISEASE         1043530 non-null  Int8    
 14  CARDIOVASCULAR    

### Check Unknown & missing value handling

In [36]:
clean_data.isna().mean().sort_values(ascending=False).head(15)

ICU               0.816377
INTUBED           0.816221
PREGNANT          0.377293
PNEUMONIA         0.015262
OTHER_DISEASE     0.004811
INMSUPR           0.003246
DIABETES          0.003183
TOBACCO           0.003071
HIPERTENSION      0.002960
CARDIOVASCULAR    0.002934
OBESITY           0.002892
RENAL_CHRONIC     0.002867
COPD              0.002864
ASTHMA            0.002841
SEX               0.000000
dtype: float64

In [39]:
clean_data['DIED'].value_counts(normalize=True)

DIED
0    0.926622
1    0.073378
Name: proportion, dtype: float64

In [48]:
for c in clean_data.columns:
    if f"{c}_unk" in clean_data.columns:
        incorrect_mask = (
            clean_data[c].isna() & (clean_data[f"{c}_unk"] != 1) |
            clean_data[c].notna() & (clean_data[f"{c}_unk"] != 0)
        )
        print(f"incorrenct percent {c}: {incorrect_mask.mean()*100}")

incorrenct percent USMER: 0.0
incorrenct percent SEX: 0.0
incorrenct percent INTUBED: 0.0
incorrenct percent PNEUMONIA: 0.0
incorrenct percent AGE: 0.0
incorrenct percent PREGNANT: 12.554657511384498
incorrenct percent DIABETES: 0.0
incorrenct percent COPD: 0.0
incorrenct percent ASTHMA: 0.0
incorrenct percent INMSUPR: 0.0
incorrenct percent HIPERTENSION: 0.0
incorrenct percent OTHER_DISEASE: 0.0
incorrenct percent CARDIOVASCULAR: 0.0
incorrenct percent OBESITY: 0.0
incorrenct percent RENAL_CHRONIC: 0.0
incorrenct percent TOBACCO: 0.0
incorrenct percent ICU: 0.0
